In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, KFold

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

In [ ]:
from sklearn.tree import DecisionTreeClassifier

from sklearn import tree

# Intro

The first part of this notebook uses a toy dataset to illustrate __how splits are made and evaluated__ when constructing a classification tree.

The example ilustrates splits done on a quantitative predictor (Income).

The example evaluates the splits based on the error rate. Notice that the CART algorithm implemented in scikit learn does not use the error rate to make splits. It uses either the Gini index or Entropy (whichever the user chooses).

__Creating the toy dataset__

There are three predictors. The outcome variable is Credit_Risk.

In [ ]:
toy_df_classif = pd.DataFrame({'Savings':['Med','Low','High','Med','Low','High','Low','Med'], 'Assets':['High','Low','Med','Med','Med','High','Low','Med'], 'Income':[75,50,25,50,100,25,25,75], 'Credit_Risk':['Good','Bad','Bad','Good','Good','Good','Bad','Good']})

In [ ]:
toy_df_classif

#### Exploring how splits are made when constructing a CT
#### Exploring splits on Income

The first thing to do before creating __the possible splits__ on a quantitative predictor is to __obtain the cutoff points__.

To obtain the possible cutoff points, I will first store all Income values (excluding duplicates) in an array called __Income_unique__.

In [ ]:
Income_unique= np.sort(toy_df_classif['Income'].unique())
Income_unique

In [ ]:
cutpoints=[]
for i in np.arange(len(Income_unique)-1):  # I need i to reach only the second to last index (not  last one). np.arange(len(x)-1) generates the numbers 0,1,2
    cutpoints.append((Income_unique[i]+ Income_unique[i+1])/2)

In [ ]:
cutpoints

The algorithm will find an split using Income based on these three cutpoints and would decide the best split (i.e., the split
leading to the purest child nodes).

The next code cells will show in details (step by step) how __to obtain and evaluate ONLY the split done at 37.5.__ Later on, you will see a loop where all three cutoff points are evaluated.

__Reminder__: The algorithm would also do all the possible splits for the other two predictors, Savings and Assets, and would actually choose the best split obtained across all three predictors and across all cutoffs tested for each predictor.

In [ ]:
# LEFT NODE FOR CUTPOINT 37.5

toy_df_classif.loc[toy_df_classif['Income'] <= 37.5, ]


In [ ]:
# RIGHT NODE FOR CUTPOINT 37.5

toy_df_classif.loc[toy_df_classif['Income']> 37.5, ]

__What's the prediction of Y for the left node?__

Answer: The prediction is Bad because it is the most common class in this node. The next two code cells show you how to get this label programmatically.

In [ ]:
toy_df_classif.loc[toy_df_classif['Income'] <= 37.5, ]['Credit_Risk'].value_counts()

In [ ]:
toy_df_classif.loc[toy_df_classif['Income'] <= 37.5, ]['Credit_Risk'].value_counts().idxmax()

__What's the error rate at the left node?__

Error rate for left node = 1/3 (proportion for the least frequent class in this node, which is Good)

In [ ]:
# With code

error_left = toy_df_classif.loc[toy_df_classif['Income'] <= 37.5, ]['Credit_Risk'].value_counts(normalize=True).min()

In [ ]:
error_left

__What's the prediction of Y for the right node?__



Answer: The prediction is Good because it is the most common class in this node.

In [ ]:
toy_df_classif.loc[toy_df_classif['Income'] > 37.5, ]['Credit_Risk'].value_counts().idxmax()

__What's the error rate at the right node?__


Error rate for right node = 1/5= 0.2 (proportion of obs that belong to the least frequent class)

In [ ]:
# With code

error_right= toy_df_classif.loc[toy_df_classif['Income'] > 37.5, ]['Credit_Risk'].value_counts(normalize=True).min()

In [ ]:
error_right

__What's the error associated with making a split at 37.5 based on Income?__

Combine the error rate from both the left and right node!

How can we combine these errors? We need to obtain the weighted average of the errors for the left and right nodes.

__Error for split at 37.5= weight for left node * error rate for left node + weight for right node * error rate for right node__

weight for left node= obs that reached the left node/ obs before the split

weight for right node= obs that reached the right node/ obs before the split

__Curiosity:__ Notice that the unweighted average (which we do not need to compute), we will be obtained:

Unweighted average = (error rate for left node + error rate for right node)/ 2 = 0.5 * error rate for left node + 0.5 * error rate for right node

In [ ]:
left_node_size= len (toy_df_classif.loc[toy_df_classif['Income'] <= 37.5, ])
left_node_size

In [ ]:
right_node_size= len (toy_df_classif.loc[toy_df_classif['Income'] > 37.5, ] )
right_node_size

In [ ]:
# Weighted error for the split

error_left * (left_node_size/(left_node_size+right_node_size)) + error_right * (right_node_size/(left_node_size+right_node_size))

Is the split benefitial? Is the error after the split lower than the error before split?

In [ ]:
# Error before the split

toy_df_classif['Credit_Risk'].value_counts(normalize= True).min()

In [ ]:
3/8

Doing an split at 37.5 is benefitial because the error rate after the split is lower than the error rate before the split.

The previous code cells included the steps to compute the weighted error rate for the split at 37.5 on Income.

The next loop summarizes the computation of the weighted error for the three splits on Income, that is the splits at 37.5, 62.5, and 87.5.

We can get what's the best of all the splits for Income based on the output of this loop.

In [ ]:
error_rate_splits =[]
for i in cutpoints:
    left_node_size= len (toy_df_classif.loc[toy_df_classif['Income'] <= i, ])
    right_node_size= len (toy_df_classif.loc[toy_df_classif['Income'] > i, ] )
    error_left= toy_df_classif['Credit_Risk'][toy_df_classif['Income']<=i].value_counts(normalize=True).min()
    error_right= toy_df_classif['Credit_Risk'][toy_df_classif['Income']>i].value_counts(normalize=True).min()
    error_rate_splits.append(error_left*(left_node_size/(left_node_size+right_node_size)) + error_right*(right_node_size/(left_node_size+right_node_size)))

In [ ]:
error_rate_splits_df= pd.DataFrame (data= error_rate_splits, index= cutpoints, columns=['Weighted_Error_Split'])

In [ ]:
error_rate_splits_df.index.name = ('Cutoff_point')

In [ ]:
error_rate_splits_df

The best split and only meaningful split is the one at 37.5 because it leads to the lowest error rate (and the only error rate lower than the error rate at the parent node).

### BACK TO THE SLIDES !

### Example comparing error rate, Gini, and Entropy.

### **YOU WILL EXPLORE THIS EXAMPLE INDEPENDENTLY !!!**

The following example will show that the Gini index and Entropy are more sensitive to changes in the node probabilities than the error rate.

- Y has two classes (0 and 1)
- The node under consideration contains 800 obs, 400 from each class.
- Split 1 leads to a left node with 300 zeros and 100 ones AND a right node with  100 zeros and 300 ones.
- Split 2 leads to a left node with 200 zeros and 400 ones AND a right node with  200 zeros and 0 ones.
- Compute the error rate, Gini, and Entropy for both splits.

Error rate split 1

Reminder: Error rate for a split=  Average weighted error

Average weighted error= Weight for left node * error rate for left node + weight for right node * error rate for right node

In [ ]:
(400/800)* (100/400) + (400/800)* (100/400)


Error rate split 2

In [ ]:
(600/800)* (200/600) + (200/800)* (0/200) # the right node for split 2 is completely pure!

According to the error rate, splits 1 and 2 are equally good.

Gini split 1

Gini index for a split= Weighted Gini for the split

Weighted Gini for the split= Weight for left node * Gini for left node + weight for right node * Gini for right node

G= 2 p1 (1-p1)

In [ ]:
(400/800)* (2*(100/400)*(300/400)) + (400/800)* (2*(300/400)*(100/400))

In [ ]:
# 1 - (100/400) = 300/400

Gini split 2

In [ ]:
(600/800)* (2*(400/600)*(200/600)) + (200/800)* (2*(0/200)*(200/200))

According to Gini, split 2 is better. Gini heavily weighs the fact that split 2 gives us a perfectly pure node.

Entropy split 1

Entropy for a split= Weighted Entropy for the split

Weighted Entropy for the split= Weight for left node * Entropy for left node + weight for right node * Entropy for right node

D= - (1-p1) * log (1 -p1) - p1 * log(p1)

Reminder: p1 for the left node in split 1 is 100/400

Reminder: p1 for the right node in split 1 is 300/400

In [ ]:
(400/800)* (-(300/400)*np.log(300/400)- (100/400)*np.log(100/400)) + \
(400/800)* (-(100/400)*np.log(100/400) -(300/400)*np.log(300/400))

Entropy split 2

D= - (1-p1) * log (1 -p1) - p1 * log(p1)

Reminder: p1 for the left node in split 2 is 400/600

Reminder: p1 for the right node in split 2 is 0

In [ ]:
(600/800)* ( -(200/600)*np.log(200/600) - (400/600)*np.log(400/600) ) + \
(200/800)* (- (1-0) * np.log (1 -0) - 0 * 0)

# Note: We are assuming that log (0) = 0. In reality, log (0) is not defined.

According to the Entropy, split 2 is better. Similarly to Gini, Entropy heavily weighs the fact that split 2 gives us a perfectly pure node

## Example 1: Obtaining a classification tree for the Default data set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
default_data_path = '/content/drive/MyDrive/CAP 4633C/Datasets_CAP4633C/Default.csv'

In [ ]:
default_df = pd.read_csv(default_data_path)

In [ ]:
default_df_dummies = pd.get_dummies(default_df,columns=['student'], drop_first= True)

In [ ]:
default_df_dummies.info()

In [ ]:
X_def = default_df_dummies.drop ('default', axis=1)

In [ ]:
y_def = default_df_dummies['default']

### Pre-pruning strategy using GridSearch.

Reminder: Pre-pruning means preventing the final tree to be too big by setting up several stopping criteria.

In [ ]:
X_train_def, X_test_def, y_train_def, y_test_def = train_test_split (X_def, y_def, test_size= 0.2, random_state= 1, stratify = y_def)

In [ ]:
# Notice that we have an extra hyperparameter here when compared to regression trees: 'criterion'.
# Therefore, I am using fewer options for each hyperparameter in comparison with regression trees to minimize comp time.

hyperparam_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': np.arange(2, 7), # depths from 2 to 6. It is rare that the best depth is > 5 anyways
    'min_samples_split':[0.05, 0.1, 0.15, 0.2],
    'min_samples_leaf':[0.05, 0.1, 0.15, 0.2],
    'min_impurity_decrease': [0, 0.0005, 0.001, 0.01, 0.05]
}

In [ ]:
cv_set_up = KFold (n_splits= 10 , shuffle= True, random_state= 1)

In [ ]:
grid_search_setting_pre_pru = GridSearchCV(DecisionTreeClassifier(random_state= 1), hyperparam_grid, cv = cv_set_up, scoring='accuracy')

In [ ]:
grid_search_setting_pre_pru.fit(X_train_def, y_train_def)

In [ ]:
print('The best hyperparameters are: ', grid_search_setting_pre_pru.best_params_)

In [ ]:
tree_default_prepruned = DecisionTreeClassifier(criterion='gini', max_depth = 2, min_samples_split = 0.05, min_samples_leaf = 0.05, min_impurity_decrease = 0, random_state=1)

Remember that you can also retrieve the best solution by simply invoking the 'best_estimator_' attribute on the grid search object. Like this:

tree_default_prepruned = grid_search_setting_pre_pru.best_estimator_

In [ ]:
tree_default_prepruned.fit(X_train_def, y_train_def)

In [ ]:
plt.figure(figsize=(8, 8))
tree.plot_tree(tree_default_prepruned, filled=True, rounded= True, feature_names=X_train_def.columns, fontsize=12)
plt.show()

In [ ]:
tree_default_prepruned.classes_

In [ ]:
y_pred_pre_tree_def = tree_default_prepruned.predict(X_test_def)

In [ ]:
confusion_matrix(y_test_def, y_pred_pre_tree_def)

Without obtaining the classification report, tell me the specificity and sensitivity of this classifier.

In [ ]:
# Specificity = Recall for 0 class ('No' class)



In [ ]:
# Sensitivity = Recall for 1 class ('Yes' class)



### Post-pruning via CCP

In [ ]:
# We are growing a big (unpruned) tree using Gini (which is the default criterion)

tree_default_unpruned = DecisionTreeClassifier(criterion='gini', random_state=1)

In [ ]:
tree_default_unpruned.fit(X_train_def , y_train_def)

In [ ]:
ccp_path= tree_default_unpruned.cost_complexity_pruning_path(X_train_def , y_train_def)

In [ ]:
hyperparam_grid_alpha = { 'ccp_alpha': ccp_path.ccp_alphas}

In [ ]:
gridSearch_alpha = GridSearchCV(tree_default_unpruned, hyperparam_grid_alpha,  cv= cv_set_up , scoring='accuracy', n_jobs= -1)

In [ ]:
gridSearch_alpha.fit(X_train_def, y_train_def)

In [ ]:
print('The best parameters are: ', gridSearch_alpha.best_params_)

In [ ]:
tree_default_postpruned= DecisionTreeClassifier( random_state=1, ccp_alpha= gridSearch_alpha.best_params_['ccp_alpha'])

In [ ]:
tree_default_postpruned.fit(X_train_def, y_train_def)

In [ ]:
plt.figure(figsize=(12,12))
tree.plot_tree(tree_default_postpruned,filled=True, rounded= True, feature_names=X_train_def.columns, fontsize=12)
plt.show()

In [ ]:
y_pred_post_pr_tree_def = tree_default_postpruned.predict(X_test_def)

In [ ]:
confusion_matrix (y_test_def, y_pred_post_pr_tree_def)

In [ ]:
print (classification_report (y_test_def, y_pred_post_pr_tree_def))

__What are the most important predictors?__

What predictors show up in the tree?

In [ ]:
X_train_def.columns [tree_default_postpruned.feature_importances_!=0]

What is the importance of each predictor?

In [ ]:
importance_df = pd.DataFrame({'Feature': X_train_def.columns, 'Importance': tree_default_postpruned.feature_importances_})

In [ ]:
importance_df.sort_values(by='Importance', ascending=False)

Obtain the ROC curve and AUC.

In [ ]:
roc_auc_postpruned = roc_auc_score(y_test_def, tree_default_postpruned.predict_proba(X_test_def)[:, 1])

In [ ]:
np.round (roc_auc_postpruned, 3)

In [ ]:
fpr, tpr, threshold = roc_curve(y_test_def, tree_default_postpruned.predict_proba(X_test_def)[:, 1], pos_label='Yes')

In [ ]:
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC ='+ str (np.round (roc_auc_postpruned, 3)))
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Compare the performance of NB and the post-pruned tree based on AUC:

NB results:

AUC= 0.951


The NB classifier gives a better balance between sensitivity and specificity across multiple probability thresholds (because it has a higher AUC).

## Example 2: Obtaining a classification tree for the Churning dataset

In [ ]:
churning_data_path= '/content/drive/MyDrive/CAP 4633C/Datasets_CAP4633C/bank_customer_churn.csv'

In [ ]:
churning_df= pd.read_csv(churning_data_path)

In [ ]:
churning_df.drop(["RowNumber", "CustomerId", "Surname"], axis = 1, inplace= True)

In [ ]:
churning_df_dummies= pd.get_dummies(churning_df,columns=['Geography', 'Gender'], drop_first= True)

In [ ]:
churning_df_dummies.info()

Obtain a tree, first by applying pre-pruning and then by applying post-pruning. Compare the results (i.e., the prediction of both trees on test data).

Only obtain the feature importance and plot the tree for one tree (the best one between the two trees).

Continue working independently.